### In this notebook:
#### Functions to eliminate duplicates that happen when the HIPPARCOS and GAIA star catalogs get combined

In [14]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import astropy.io.ascii as a
import astropy.table
import FunctionsToLoadGaia as load
import FunctionsForGaia as f
from math import pi

In [32]:
def extractRaDec(filename):
    
    hdu = astropy.io.fits.open(filename)
    table = astropy.table.Table(hdu[1].data)
    dec = (table['dec'].data)*(pi/180) #change degrees to radians.
    
    RA = (table['ra'].data)
    over180 = RA > 180 #want RA values to be from -180 to 180
    RA[over180] = (RA[over180] - 360)
    ra = RA*(pi/180) #change degrees to radians.
    
    return ra, dec

In [33]:
from glob import glob
filenames = glob('data/TgasSource_000-000-*.fits')

RA,DEC = np.array([]), np.array([])
for f in filenames:
    ra,dec = extractRaDec(f)
    RA = np.append(RA,ra)
    DEC = np.append(DEC,dec)

GaiaRaDec=astropy.table.Table([RA,DEC], names=['RightAscension','Declination'])

In [4]:
def findRaDec(Hipparcos):
    return HipRaDec

In [34]:
def findDuplicates(HipRaDec,GaiaRaDec):
    """
    This function will go over the arrrays of stars from the HIPPARCOS and GAIA
    catalogs and determine which stars are duplicates
    
    Parameters:
    ----------------------------------
    GaiaRaDec = array of the RA and DEC of the GAIA catalog
    HipRaDec = array of the RA and DEC of the HIPPARCOS catalog
    
    Returns:
    -----------------------------------
    duplicates = array of the 'positions' of the stars in the GAIA arrays that 
    are repeats of the HIPPARCOS catalog
    """
    
    for i in range(len(original)):
        ora=HipRaDec['RA'][i]
        odec=HipRaDec['DEC'][i]
        distance = (GaiaRADec-ora)**2
        match = distance<threshold
        ioverlap = np.nonzero(match)
        match=[F,F,F,F]
        np.nonzero(match)=[]
    
    
    return duplicates

SyntaxError: can't assign to function call (<ipython-input-34-690c6a806d0e>, line 24)

In [3]:
def eliminateDuplicates(duplicates,Gaia):
    """
    This function eliminates the stars on the GAIA array tht are repeated on the 
    hipparcos catalog
    
    Parameters:
    ------------------------------------
    duplicates = array of the 'positions' of the duplicates in the gaia catalog
    Gaia= 
    
    Returns:
    -------------------------------------
    GaiaNoDuplicates: array of the positions of the stars from the Gaia catalog
    without the stars that HIPPARCOS already mapped
    """
    
    return GaiaNoDuplicates